## Pic_Zoom

- 1. opencv 的图片缩放
- 2. 插值方法
- 3. matScale + warpAffine()


#### 1. opencv 的图片缩放

---


In [ ]:
# 1 load 2 info 3 resize 4 check
import cv2
img = cv2.imread('Album.jpg', 1)
imgInfo = img.shape  # h * w * channels (i.e.: RBG, 3 channels)
print(imgInfo)
height = imgInfo[0]  # int
width = imgInfo[1]
mode = imgInfo[2]

# 1 放大/缩小 2 等/非等比例缩放 2:3

# 等比例: 乘上相同系数
dstHeight = int(height * 0.5)
dstWidth = int(width * 0.5)

dst = cv2.resize(img, (dstWidth, dstHeight))
cv2.imshow('image', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 2. 插值方法
- 插值方法：最近邻域插值 双线性 像素关系重采样 立方插值<br />


- 目的：src缩小为dst，dst(x,y) <- src(newX,newY)<br />
- 公式：
```
newX= x*(src行数/dst行数)
newY= y*(src列数/dst列数)
```


- 例子：求某一点dst(?,?)，根据公式求出应该取原图的(newX,newY)= (15.2,22.3) <br />

<img src="pic_Markdown/zoom1.png" width="30%">

```
1) 最近邻域插值：(15.2,22.3) = (15,22) 
2) 双线性: 需要用到src(15.2,22.4)周围四点的值
    A1 A2 B1 B2坐标:
    A1(15.2,22)  A2(15.2,23) : 橘蓝交点
    B1(15,22.3)  B2(16,22.3) ：绿蓝交点
    
    A1 A2 B1 B2像素值计算:
    # A1 = 20%上方 + 80%下方 ,A2同理
    # B1 = 30%左方 + 70%右方 ,B2同 
    
    最终点像素值：
    # 1）最终点 = 30%A1 + 70%A2 
    # 2）最终点 = 20%B1 + 80%B2
```


In [ ]:
# src 10*20 ,dst 5*10

# 最近邻域插值
# e.g:
#      求dst(1,2) 该点像素值
#  newX= x*(src行数/dst行数)= 1 *（10/5)=2
#  newY= y*(src列数/dst列数)= 2 * (20/10)=4
#      so dst(1,2) <- src(2,4)
# 如果(newX,newY)为小数  12.3 = 12 四舍五入

# 双线性插值
# A1 = 20%上方 + 80%下方 ,A2同理
# B1 = 30%左方 + 70%右方 ,B2同
# 1）最终点 = 30%A1 + 70%A2
# 2）最终点 = 20%B1 + 80%B2

import cv2
import numpy as np
# 1 img_info 2 空白模板 3 xy 4 注意数据类型
# 邻近插值法
img = cv2.imread('Album.jpg', 1)
imgInfo = img.shape
height = img.shape[0]  # int
width = img.shape[1]
dstHeight = int(height / 2)  # int/int = float
# int//int = int
dstWidth = int(width / 2)  # dstWidth = width//2 也OK
dstImage = np.zeros((dstHeight, dstWidth, 3), np.uint8)  # 0-255
for i in range(0, dstHeight):  # 高度 -> 行
    for j in range(0, dstWidth):  # 宽度 列
        iNew = int(i * (height * 1.0 / dstHeight))  # *1.0 保证float
        jNew = int(j * (width * 1.0 / dstWidth))
        dstImage[i, j] = img[iNew, jNew]
cv2.imshow('dstImage', dstImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 3. matScale 缩放矩阵
自己定义一个matScale,配合cv2.warpAffine(img,matScale,img.shape)来实现图片缩放<br />
matScale：

$$
 \begin{bmatrix}
   A1 & A2 & B1 \\
   A3 & A4 & B2 
  \end{bmatrix} 
$$

$$newX=A1*x+A2*y +B1\\
newY=A3*x+A4*y+B2
$$

所以如果缩放0.5倍：x->x\*0.5, y->y\*0.5<br />
即：newX = 0.5\*x  <br />
则可得到$A1=0.5, A2=B1=0$，这样来定义matScale


In [1]:
# matScale=[[A1 A2 B1],[A3 A4 B2]]
# newX = A1 * x + A2 * y + B1
# newY = A3 * x + A4 * y + B2
# newX = 0.5 * x , 则A1=0.5

import cv2
import numpy as np

img = cv2.imread('Album.jpg', 1)
imgInfo = img.shape
height = img.shape[0]  # int
width = img.shape[1]
# 缩放矩阵 // 图片移位也可以定义一个移位矩阵，用warpAffine()实现移位
matScale=np.float32([[0.5,0,0],[0,0.5,0]])
dst=cv2.warpAffine(img,matScale,(width//2,height//2))
cv2.imshow('dstImage',dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

107